In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import html2text
import httpx
import json
import lxml
import re
import glob

from lxml import html
from extractcontent3 import ExtractContent
from readability.readability import Document

from tqdm import tqdm

In [3]:
def extractcontent(html):
    extractor = ExtractContent()

    opt = {"threshold":50}
    extractor.set_option(opt)

    extractor.analyse(html)
    text, title = extractor.as_text()
    html, title = extractor.as_html()
    title = extractor.extract_title(html)
    return text

def readability(html):
    doc = Document(html)
    title = doc.title()
    summary = doc.summary()
    #text1 = html2text.html2text(article)
    summary_text = lxml.html.fromstring(summary).text_content().strip()
    #return text1 + '\n---\n' + text2
    return title, summary_text

In [7]:
!ls ../../data/fc2b/html

1  2  3


In [8]:
headers = {}
cookies = {'age_check': '1'}

In [9]:
toxicity_genres = (333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 378, 379, 381)

In [10]:
genre_list = httpx.get('https://blog.fc2.com/genrelist/', headers=headers, cookies=cookies)
genre_groups = html.fromstring(genre_list.text).xpath('//*[@class="genre_group"]')
genre_data = []
for group in genre_groups:
    genre = group.xpath('./h3/a')[0]
    genre_id = genre.attrib['href'].strip().split('/')[-2]
    genre_text = genre.text.strip()
    
    children = []
    subgenres = group.xpath('./ul/li/a')
    for node in subgenres:
        subgenre_id = re.findall(r'/subgenre/([0-9]+)/', node.attrib['href'])[0]
        subgenre_text = ''.join(node.itertext()).strip()
        children.append({
            'id': subgenre_id,
            'name': subgenre_text
        })

    genre_data.append({
        'id': genre_id,
        'name': genre_text,
        'subgenre': children
    })
    
subgenre2genre = {subgenre['id']: (genre['id'], genre['name']) for genre in genre_data for subgenre in genre['subgenre']}
subgenre2name = {subgenre['id']: subgenre['name'] for genre in genre_data for subgenre in genre['subgenre']}

In [15]:
data = []
for path in tqdm(glob.glob('../../data/fc2b/**/*.html', recursive=True)[:10]):
    entry = path.split('/')[-1]
    subgenre_id = path.split('/')[-2]
    toxicity = 1 if genre_id in toxicity_genres else 0
    try:
        title, summary = readability(open(path).read())
        if summary.strip():
            data.append({
                'blog': entry.split('#')[0],
                'entry': entry.split('#')[1],
                'genre_id': subgenre2genre[subgenre_id][0],
                'genre_name': subgenre2genre[subgenre_id][1],
                'subgenre_id': subgenre_id,
                'subgenre_name': subgenre2name[subgenre_id],
                'toxicity': toxicity,
                'title': title,
                'text': summary
            })
    except Exception as exc:
        print(f"{type(exc).__name__}. {str(exc)} {path}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.36it/s]

ParserError. Document is empty ../../data/fc2b/html/1/kzkobo.blog.fc2.com#blog-entry-5819.html
ParserError. Document is empty ../../data/fc2b/html/1/gayaminami.blog.fc2.com#blog-entry-483.html


In [16]:
data[0]

{'blog': '5ball.blog113.fc2.com',
 'entry': 'blog-entry-1052.html',
 'genre_id': '7',
 'genre_name': '日記',
 'subgenre_id': '1',
 'subgenre_name': '小学生',
 'toxicity': 0,
 'title': 'シンクロニシティのジャグリング教室 - 11月26日 18:30 910回目',
 'text': 'Author:シンクロニシティ\nチャチャとラガーの二人組。ショーも見に来てね！'}

In [9]:
with open('/tmp/fc2b.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True)